# Shors Algorithm

## Task 1: A classical implementation

First, we need to check special cases for Shor's Algorithm

### 1.1 IsEven

In [ ]:
%kata IsEven_Test

operation IsEven(N : Int) : Bool {
    return (N%2==0);
}

### 1.2 IsPrime

In [ ]:
%kata IsPrime_Test

open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;

operation IsPrime(N : Int) : Bool {
    if (N <= 3) {
        return (N > 1);
    }
    if (N%2 == 0) or (N%3 == 0) {
        return false;
    }
    let sqrtN = Truncate(Sqrt(IntAsDouble(N)));
    for i in 5..6..sqrtN {
        if (N%i == 0) or (N%(i+2) == 0) {
            return false;
        }

    }
    return true;
}

GCD operation

In [ ]:
%kata GreatestCommonDivisor_Test

open Microsoft.Quantum.Math;
operation GreatestCommonDivisor (a : Int, N : Int) : Int {
    let min = Min([a,N]);
    mutable gcd = 1;
    for i in 2..min {
        if (a%i == 0 and N%i == 0) {
            set gcd = i;
        }
    }
    return gcd;
}

With these two tests, we can confirm that N is a product of two distinct prime numbers, so $N = pq$.
// Explain order finding

### 1.3 Classical order finding

In [ ]:
operation FindOrderClassicalHelper(a : BigInt, N: BigInt) : Int {
    mutable power = 0;
    repeat {
        set power += 1;
    } until (a^power % N == 1L);
    return power;
}

In [ ]:
%kata FindOrderClassical_Test

open Microsoft.Quantum.Convert;

operation FindOrderClassical(a : Int, N : Int) : Int{
    return FindOrderClassicalHelper(IntAsBigInt(a),IntAsBigInt(N));
}

### 1.4 Generate random number

In [ ]:
%kata GenerateRandomNumber_Test

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Math;

operation GenerateRandomNumber(N : Int) : Int {
    use register = Qubit[BitSizeI(N-2)];
    mutable result = 0;
    repeat {
        ApplyToEachCA(H,register);
        set result = MeasureInteger(LittleEndian(register));
    } until ((result + 2) < N);
    return result+2;
}

### 1.5 General Case

In [ ]:
%kata GeneralCase_Test

operation GeneralCase(OrderFinder : ((Int, Int)=>Int), N : Int) : (Int, Int) {
    mutable result = 0;
    repeat {
        let a = GenerateRandomNumber(N);
        let gcd = GreatestCommonDivisor(a,N);
        if (gcd > 1) {
            return (gcd, N/gcd);
        }
        let r = OrderFinder(a,N);
        if (IsEven(r)) {
            let x = (a^(r/2) - 1) % N;
            let gcdX = GreatestCommonDivisor(x,N);
            if (gcdX > 1) {
                set result = gcdX;
            }
        }
    } until (result != 0);
    return (result, N/result);
}

### 1.6 Full Shor's Implimentation

In [ ]:
%kata ShorsAlgorithm_Test

operation ShorsAlgorithm(OrderFinder : (Int, Int)=> Int, N : Int) : (Int, Int) {
    if IsEven(N) {
        return (2, N/2);
    }
    if IsPrime(N) {
        fail IntAsString(N) + " is prime, so doesn't have factors.";
    }
    return GeneralCase(OrderFinder, N);
}

### 1.7 Classical Shor's Implimentation

In [ ]:
operation ShorsAlgorithmClassical(N : Int) : (Int, Int) {
    return ShorsAlgorithm(FindOrderClassical, N);
}

In [ ]:
%simulate ShorsAlgorithmClassical N=27

## Part 2: Quantum Implementation of Order Finding
### 2.1 Oracle

In [4]:
%kata OrderFindingOracle_Test

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Math;

ope

operation OrderFindingOracle(a : Int, N : Int, power : Int, target : Qubit[]) : Unit is Adj+Ctl {
    MultiplyByModularInteger(ExpModI(a, power, N), N, LittleEndian(target));
    DumpMachine();
}

/snippet_.qs(7,5): error QS5022: No identifier with the name "DumpMachine" exists.
